<h1>Benchmarking code and results for the new implemented Virtual Destillation error mittigation method</h1>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# general imports 
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
import cirq, cirq_google
import mitiq
from mitiq import QPROGRAM, MeasurementResult
import time
import pickle
from mitiq.zne.zne import execute_with_zne
from typing import Sequence, List, Iterable

In [ ]:
# importing our VD implementation

from vd import execute_with_vd
from vd.tests.testing_funcs import progressBar


In [ ]:
def present_in_plot(measurement_list, reference_value, observable_name:str ="Z", Y_vals_info=[], rel_err_on_Y=True , title="n qubits randomised identity circuit benchmark"):
    

    X, *Ys = list(zip(*measurement_list))
    xmin, xmax = 0.91 * np.min(X), 1.1 * np.max(X)

    assert len(Ys) == len(Y_vals_info), "mismath Ys and Y-infos"

    fig, ax = plt.subplots()
    fig.set_size_inches(8,6)
    hline_label = "random guesing error" if rel_err_on_Y else "true value"
    ax.axhline(reference_value, xmin, xmax, color='blue', lw="1", ls="-.", label=hline_label)

    for i, Y in enumerate(Ys):
        ax.plot(X, Y, label=Y_vals_info[i], lw=1, ms=4, marker='^')




    ax.set_xlabel("Expected # of errors")
    ax.set_xscale('log')
    if rel_err_on_Y:
        ax.set_yscale('log')
        ax.set_ylabel(f"square root distance of $\\langle {observable_name}_i \\rangle $ vector to true state")
    else:
        ax.set_ylabel(f"$\\langle {observable_name}_1 \\rangle $ of the first qubit")
        ax.set_ylim(-1.1, 1.1)
    ax.set_xlim(xmin, xmax)
    ax.legend()
    ax.set_title(title)

    return fig


In [ ]:
def create_randomised_benchmarking_circuit(N_qubits, depth ,entanglement=True, seed=None):
    single_qubit_gates = np.array([cirq.XPowGate, cirq.YPowGate, cirq.ZPowGate])

    qubits = [cirq.LineQubit(i) for i in range(N_qubits)]
    
    if seed == None:
        RNG = np.random.default_rng()
    else:
        RNG = np.random.default_rng(seed)
    
    operations = []
    entanglement_gate_count = 0
    entanglement_offset = 0

    for l in range(depth + 1):
        layer_gate_numbers = RNG.choice(3, N_qubits) # N_qubit numbers of 0 to 2, identifying one of the s.q. gates from the list for each qubit
        layer_gate_exponents = RNG.choice([0.5, 1.0], N_qubits) # determine the exponent for each gate, so we apply eithet X or X^1/2, resp. for Y and Z
        for i in range(N_qubits):
            gate = single_qubit_gates[layer_gate_numbers[i]]
            operations.append(gate(exponent=layer_gate_exponents[i]).on(qubits[i], ))

        # add syncamore gates, alternating between left and right neighbours
        if entanglement and l < depth:
            for i in range(entanglement_offset, N_qubits, 2):
                if i+1 < len(qubits): 
                    operations.append(cirq_google.ops.SycamoreGate().on(qubits[i], qubits[i+1]))
            entanglement_offset = 1 - entanglement_offset # alternate between 0 and 1


    pure_rand_qc = cirq.Circuit(operations)


    return pure_rand_qc, len(operations), entanglement_gate_count

In [ ]:
def expectation_Zi(circuit):
    """Returns Tr[ρ Z] where ρ is the state prepared by the circuit
    with depolarizing noise."""
    # print("call: ")

    # density matrix
    dm = cirq.DensityMatrixSimulator().simulate(circuit).final_density_matrix 
    
    # print(f"debug Zi:\ndm shape: {dm.shape}")
    
    n_qubits = cirq.num_qubits(circuit)
    Z = np.array([[1, 0], [0, -1]])

    Zi_operators = np.array([ np.kron(np.kron(np.eye(2**i), Z), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits)])

    # print(Zi_operators)

    Zi = np.trace(Zi_operators@ dm, axis1=1, axis2=2)
    # print()
    if not np.allclose(Zi.real, Zi, atol=1e-6):
        print("Warning: The expectation value contains a significant imaginary part. This should never happen.")
        print(Zi)
        return Zi
    else:
        return Zi.real

In [ ]:
def density_simulation_executor(circuit: QPROGRAM) -> np.ndarray:
    # density matrix
    dm = cirq.DensityMatrixSimulator().simulate(circuit).final_density_matrix 
    return dm

In [ ]:
def measurement_simulation_executor(circuit: QPROGRAM, reps = 10) -> List[MeasurementResult]:

    if isinstance(circuit, Iterable) and not isinstance(circuit, cirq.Circuit):
        circuit = circuit[0]

    # T0 = time.time()
    cirq_result = cirq.sample(circuit,repetitions=reps)
    # print(cirq_result)

    # T1 = time.time()
    result = cirq_result.measurements
    measurements = []
    
    sorted_keys = []
    for key in result:
        sorted_keys.append(int(key))
    sorted_keys.sort()

    for key in sorted_keys:
        measurements.append(result[str(key)])
    
    measurements = np.squeeze(measurements, axis=2).T
    # T2 =time.time()

    # print(f"{'> Sampling step':30s} ~ {T1-T0:8.3f} s     {(T1-T0)/(T2-T0):8.4%}")
    # print(f"{'> Reordering step':30s} ~ {T2-T1:8.3f} s     {(T2-T1)/(T2-T0):8.4%}")


    # print(measurements.tolist())



    return measurements.tolist()

In [ ]:
def vector_norm_distance(V, W):
    return np.linalg.norm((V - W))

<h2>Code to create benchmarking plots </h2> 
<h4>Modify fo suit the desired plotting data</h4>
Make sure to edit the name, qubit number, layer number, entanglement or not (in both circuit creation functions) and the K values correctly, optionally, also dont forget to set the ticks for the progress bar.

In [ ]:
# Running and saving a benchmarking iteration

import pickle

# Settings
N_qubits=6
N_layers=100

rho, gate_count, entangle_gate_count = create_randomised_benchmarking_circuit(N_qubits, N_layers, entanglement=True)
# (re)create a random circuit until at least one expectation value is nonzero
true_Zi = expectation_Zi(rho) # Fault tolerant quantum computer
while np.all(true_Zi == 0.+0.j):
    print("nope")
    rho, gate_count, entangle_gate_count = create_randomised_benchmarking_circuit(N_qubits, N_layers, entanglement=True)
    true_Zi = expectation_Zi(rho) # Fault tolerant quantum computer

print(rho)
dist_true_Zi = 0.0

BMrun = {
    "benchmark_name": "Benchmark 6q100L entangled vd density & measurement sim",
    "observable": "Z",
    "N_qubits":N_qubits,
    "N_layers":N_layers,
    "rho":rho,
    "gate_count":gate_count,
    "entangle_gate_count":entangle_gate_count
}

assert input("Is the name correctly changed? [N/y] ") == 'y', "Aborted, please change the name."



pBar = progressBar(1 + 1 + 40 * (101 + 1*(1_000 + 100)))
pBar.addTicks(1)

datapoints = []
datapoint_labels = []

BMrun["true_Zi"] = true_Zi


for i, N_exp_Err in enumerate(np.logspace(-2, 1, base=10, num=40)):

    noise_level = N_exp_Err / gate_count
    noisy_rho = rho.copy().with_noise(cirq.depolarize(p=noise_level))
    # print(noisy_rho)

    noisy_Zi = expectation_Zi(noisy_rho) # Noisy quantum computer
    dist_noisy_Zi = vector_norm_distance(true_Zi, noisy_Zi)

    # print(N_exp_Err, true_Zi, noisy_Zi)
    pBar.addTicks(1)

    measurement_list = [N_exp_Err, dist_noisy_Zi]
    datapoint_labels = ["Noisy"]

    density_vd_Zi = execute_with_vd(noisy_rho, density_simulation_executor, 2, K) # Noisy quantum computer + virtual distillation
    density_dist_vd_Zi = vector_norm_distance(true_Zi, density_vd_Zi)
    measurement_list.append(density_dist_vd_Zi)
    datapoint_labels.append(f"density_vd")
    pBar.addTicks(100)

    for K in [100_000, 1_000_000]:
        m_vd_Zi = execute_with_vd(noisy_rho, measurement_simulation_executor ,2, K) # Noisy quantum computer + virtual distillation
        m_dist_vd_Zi = vector_norm_distance(true_Zi, m_vd_Zi)
        measurement_list.append(m_dist_vd_Zi)
        datapoint_labels.append(f"measure_vd: {K=}")
        pBar.addTicks(K//1000)

    # ZNE_Zi = execute_with_zne(noisy_rho, expectation_Zi)  
    # dist_ZNE_Zi = vector_norm_distance(true_Zi, ZNE_Zi)
    # measurement_list.append(dist_ZNE_Zi)
    # datapoint_labels.append(f"ZNE - default")
    # pBar.addTicks(1)

    datapoints.append( tuple(measurement_list) )
    

BMrun["datapoints"] = datapoints
BMrun["datapoint_labels"] = datapoint_labels

with open( BMrun["benchmark_name"].replace(" ", "_") + '.pkl', 'wb') as f:
    pickle.dump(BMrun, f)

pBar.finished()




<h4>And the code snippet to actually make the plot of the just run benchmarking</h4>

In [ ]:
dist_random_state = vector_norm_distance(BMrun["true_Zi"], np.zeros(BMrun["N_qubits"]))

BMrun_fig = present_in_plot(BMrun["datapoints"], dist_random_state, BMrun["observable"], BMrun["datapoint_labels"], title=BMrun["benchmark_name"])

In [ ]:
# dictionary for shorthand names of certain plots
BM_runs = {

    "test":"Test_disjoint_BM_1q.pkl",
    "6q_entangled":"Benchmark_6q_entangled_random_circuit.pkl",
    "6q_non-entangled":"BM_6q_non_entangled.pkl"
}

In [ ]:
# function to create a plot from the pickle files 
def plot_BM_run(file_name):

    with open(file_name, 'rb') as f:
        BM_dict = pickle.load(f)
    
    dist_random_state = vector_norm_distance(BM_dict["true_Zi"], np.zeros(BM_dict["N_qubits"]))

    BM_fig = present_in_plot(BM_dict["datapoints"], dist_random_state, BM_dict["observable"], BM_dict["datapoint_labels"], title=BM_dict["benchmark_name"])  

    return BM_fig

In [ ]:
# Calling the plotting function based on the shorthands defined in the dictionary above
testrun = plot_BM_run(BM_runs['6q_entangled'])

testrun.show()

<h2>Appendix: Code snippets for timing the function implementations</h2>

In [ ]:
# Comparing the timed execution of 
# the denisty matrix and measurement based executors
test_circ_2, *args = create_randomised_benchmarking_circuit(6, 100, True, 2475257469243)
print(test_circ_2)
test_circ_2_noisy = rho.copy().with_noise(cirq.depolarize(p=0.01))

test_pBar = progressBar(1 + 1 + 100 + 100)

test_circ_Z = expectation_Zi(test_circ_2)
test_pBar.addTicks(1)
test_circ_2_noisy = test_circ_2.copy().with_noise(cirq.depolarize(p=0.1))

test_circ_Z_noisy = expectation_Zi(test_circ_2_noisy)
test_pBar.addTicks(1)
test_circ_2_vd_density_Z =  execute_with_vd(test_circ_2_noisy, density_simulation_executor, display_performance=True)
test_pBar.addTicks(100)
test_circ_2_vd_measurement_Z =  execute_with_vd(test_circ_2_noisy, measurement_simulation_executor, K=1_000_000, display_performance=True)
test_pBar.addTicks(100)
test_pBar.finished()

print(test_circ_Z)
print(test_circ_Z_noisy, vector_norm_distance(test_circ_Z, test_circ_Z_noisy))
print(test_circ_2_vd_density_Z, vector_norm_distance(test_circ_Z, test_circ_2_vd_density_Z))
print(test_circ_2_vd_measurement_Z, vector_norm_distance(test_circ_Z, test_circ_2_vd_measurement_Z))

In [ ]:
# Code snippet for benchmarking the execution time of the 
# density matrix executor implementation, as a function of 
# the number of qubits.
Zi_lis = []
times_lis = []
for n in range(1, 7):
    rho, *args = create_randomised_benchmarking_circuit(n, 100, True, 2475682256243)
    noisy_rho = rho.copy().with_noise(cirq.depolarize(p=0.01))
    T0 = time.time()
    Zi = execute_with_vd(noisy_rho, density_simulation_executor, display_performance=True)
    T1 = time.time()
    Zi_lis.append(Zi)
    times_lis.append(T1-T0)
    print(f"{n} qubits --> {T1-T0:10.5f} sec runtime")


In [ ]:
# Code snippet for benchmarking the execution time of the 
# measurement executor implementation, as a function of 
# the number of qubits, or the number of shots K.

Zi_lis_m = []
times_lis_m = []
for k in [10, 100, 500, 1000, 5000, 10000, 100_000, 1_000_000]:
    rho, *args = create_randomised_benchmarking_circuit(4, 100, True, 2475682256243)
    noisy_rho = rho.copy().with_noise(cirq.depolarize(p=0.01))
    T0 = time.time()
    Zi = execute_with_vd(noisy_rho, measurement_simulation_executor,K=k ,display_performance=True)
    T1 = time.time()
    Zi_lis_m.append(Zi)
    times_lis_m.append(T1-T0)
    # print(f"{n} qubits --> {T1-T0:10.5f} sec runtime")
    print(f"K = {k:10d} --> {T1-T0:10.5f} sec runtime")

# Quick note: @100 layers entangled, we have 3q->1.5s, 4q->1.3s!, 5q->2.8s, 6q->6.6s, 7q->160s, with K=100 
